# Graph Puzzle

The following graph puzzle is entirely a classical computational problem, but has relevance to parts of quantum error correction.
This notebook describes the problem in a slightly simplified way without using any confusing quantum information language.


## Setting the stage for the puzzle

Consider a periodic graph defined as coordinates over edges (`e`) and vertices (`V`) that are arranged in a `D` by `D` grid structure like so:

In [215]:
from src.classical.periodic_grid_graph import PeriodicGridGraph

graph = PeriodicGridGraph(5)
graph.draw_graph()


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



This example has a dimension of `D = 5`, and indexes run from left-to-right, top-to-bottom, for both edges and vertices.

The graph has periodic boundary conditions, where the left/right and top/bottom set of edges are identical. In other words, the graph "wraps around" like a pacman grid - if you travel beyond the top of the grid you will re-emerge at the bottom, and vice versa. Same goes for left/right.

You can see here that every edge connects 2 adjacent vertices, and every vertex has a degree of 4 (i.e. is incident to 4 edges).

Here are the rules of the graph:

- both edges and vertices can exist in one of two states: `0` or `1`;
- if an edge is in the state `1`, then it flips the state of its 2 adjacent vertices;

Let's take a look at the same graph again, where all edges and vertices are in the `0` state:

In [216]:
graph.draw_graph()


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



Let's see what happens when we flip the state of edge `e17`:

In [217]:
edges_to_flip = [17]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)



        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



We see here that by flipping edge `e17` we have also flipped the state of vertices `V6` and `V7` from `0` to `1`.

What happens if we also flip edge `e22`? Let's have a look:

In [218]:
edges_to_flip = [17, 22]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



We see here that by flipping edges `e17` and `e22` we have also flipped the state of vertices `V6` and `V12` from `0` to `1`.

But since both `e17` and `e22` have each flipped the state of `V7`, that vertex goes back to the original `0` state.
This introduces the concept of the flipped edge "chain": given a chain of adjacent flipped edges, only vertices at the ends of the chain will be flipped.

From all this, we see that <u>flipped vertices always appear in pairs</u>.

What happens when we flip the states of edges `e21` and `e27`?

In [219]:
edges_to_flip = [21, 27]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



We see that this has resulted in flipping the exact same vertices that edges `e17` and `e22` did!

A similar thing can happen if the flipped edge chain crosses one of the boundaries.
Consider what happens when we flip edges `e25`, `e26` and `e27`:

In [220]:
edges_to_flip = [25, 26, 27]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



The eagle-eyed will notice the same vertices would be flipped if edges `e28` and `e29` were flipped. In fact, there are quite a few edge chains that would do this, for example:

In [221]:
edges_to_flip = [6, 7, 8, 9, 10, 14, 16, 17, 22, 24]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



We see here that many chains can be equivalent with respect to the vertices that are ultimately flipped.

Of course, edges flipped on different areas of the graph will result in even more flipped vertex pairs:

In [222]:
edges_to_flip = [8, 9, 14, 20, 24, 26]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



What happens when an edge chain goes from end-to-end either top/bottom or left/right? Let's see:

In [223]:
edges_to_flip = [1, 7, 12, 17, 21, 31, 41]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3              e4      


e5      V0      e6      V1      e7      V2      e8      V3      e9      V4      e5      


        e10             e11             e12             e13             e14     


e15     V5      e16     V6      e17     V7      e18     V8      e19     V9      e15     


        e20             e21             e22             e23             e24     


e25     V10     e26     V11     e27     V12     e28     V13     e29     V14     e25     


        e30             e31             e32             e33             e34     


e35     V15     e36     V16     e37     V17     e38     V18     e39     V19     e35     


        e40             e41             e42             e43             e44     


e45     V20     e46     V21     e47     V22     e48     V23     e49     V24     e45     


        e0              e1              e2              e3              e4      




D = 5 periodic grid graph



No vertices are in the `1` state! For now this isn't too important, but is interesting to note.


## The puzzle itself

Now we have defined the structure of the problem, an interesting question arises: given a grid which has some of its vertices flipped, can you determine the shortest edge chains that connect pairs of the flipped vertices?

In other words, how do we set the state of all flipped vertices back to `0` by flipping the smallest number of edges?

For example, consider this grid:

In [224]:
graph = PeriodicGridGraph(6)
flipped_vertices = [3, 12, 15, 19]
graph.draw_graph(vertices=flipped_vertices)


        e0              e1              e2              e3              e4              e5      


e6      V0      e7      V1      e8      V2      e9      V3      e10     V4      e11     V5      e6      


        e12             e13             e14             e15             e16             e17     


e18     V6      e19     V7      e20     V8      e21     V9      e22     V10     e23     V11     e18     


        e24             e25             e26             e27             e28             e29     


e30     V12     e31     V13     e32     V14     e33     V15     e34     V16     e35     V17     e30     


        e36             e37             e38             e39             e40             e41     


e42     V18     e43     V19     e44     V20     e45     V21     e46     V22     e47     V23     e42     


        e48             e49             e50             e51             e52             e53     


e54     V24     e55     V25     e56     V26     e57     V27     e58     V28 

Here, the shortest path between vertices `V3` and `V15` is via edges `e15` and `e27`.

For vertices `V12` and `V19` there are two equivalent paths: via edges `e30` and `e36` or edges `e26` and `e31`.

There are other paths that connect different vertex pairs, but not of these is shorter than the ones described already.

What if we add a few more vertices?

In [225]:
flipped_vertices = [0, 3, 5, 12, 15, 19]
graph.draw_graph(vertices=flipped_vertices)


        e0              e1              e2              e3              e4              e5      


e6      V0      e7      V1      e8      V2      e9      V3      e10     V4      e11     V5      e6      


        e12             e13             e14             e15             e16             e17     


e18     V6      e19     V7      e20     V8      e21     V9      e22     V10     e23     V11     e18     


        e24             e25             e26             e27             e28             e29     


e30     V12     e31     V13     e32     V14     e33     V15     e34     V16     e35     V17     e30     


        e36             e37             e38             e39             e40             e41     


e42     V18     e43     V19     e44     V20     e45     V21     e46     V22     e47     V23     e42     


        e48             e49             e50             e51             e52             e53     


e54     V24     e55     V25     e56     V26     e57     V27     e58     V28 

It's tempting to say that vertices `V3` and `V5` could be paired up via edges `e10` and `e11`, leaving `V15` to be paired with some other vertex.

But if we recall the periodic boundary conditions then we see that `V5` can actually be flipped by a shorter path to `V0` via edge `e6`:

In [226]:
flipped_vertices = [0, 3, 5, 12, 15, 19]
flipped_edges = [6, 15, 27, 36, 43]
graph.draw_graph(vertices=flipped_vertices, edges=flipped_edges)


        e0              e1              e2              e3              e4              e5      


e6      V0      e7      V1      e8      V2      e9      V3      e10     V4      e11     V5      e6      


        e12             e13             e14             e15             e16             e17     


e18     V6      e19     V7      e20     V8      e21     V9      e22     V10     e23     V11     e18     


        e24             e25             e26             e27             e28             e29     


e30     V12     e31     V13     e32     V14     e33     V15     e34     V16     e35     V17     e30     


        e36             e37             e38             e39             e40             e41     


e42     V18     e43     V19     e44     V20     e45     V21     e46     V22     e47     V23     e42     


        e48             e49             e50             e51             e52             e53     


e54     V24     e55     V25     e56     V26     e57     V27     e58     V28 

So, can we come up with an efficient algorithm that helps solves this puzzle?

There's a few ways to do this...